# Model the Lines Importance as Charge on each Line

In [ ]:
# import the libraries needed to represent the tsp and to embed it on the quantum annealer
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from neal import SimulatedAnnealingSampler

import numpy as np
import dimod
import itertools 
import minorminer
import networkx as nx
import dwave_networkx as dnx


In [ ]:
# problem parameters
PROBLEM_NAME = 'tsp4.txt'
TOKEN = ''
CHAIN_STRENGTH = 2.0

D = np.loadtxt('Test_Problems/last_tests/' + PROBLEM_NAME, dtype='i', delimiter=' ')
Q = [80, 80, 40, 40, 20, 20, 10, 10]

n = D.shape[0]
num_reads = 10000    # num_reads must be between [1, 10000] for execution on QPU

print(f'D is: \n{D}')
print(f'Q is: \n{Q}')

In [ ]:
# normalization factors

# importance of constraints are set by using A,B,C in this way:
# 0 < C * max(Qv - Qu)^2 < A
# 0 < B * max(Duv) < C

# A_NORMALIZATION = 1
# C_NORMALIZATION = A_NORMALIZATION /  (np.max(Q) - np.min(Q)**2)
# B_NORMALIZATION = C_NORMALIZATION / (np.max(D) + 1)

# B_NORMALIZATION = 1
# C_NORMALIZATION = (B_NORMALIZATION * np.max(D)) + 1
# A_NORMALIZATION = C_NORMALIZATION * (np.max(Q) - np.min(Q))**2 + 1

A_NORMALIZATION = 1
B_NORMALIZATION = 0.001
C_NORMALIZATION = 0.9
D_NORMALIZATION = 0.00025

print(A_NORMALIZATION, B_NORMALIZATION, C_NORMALIZATION, D_NORMALIZATION)

In [ ]:
from pyqubo import Binary, Constraint, Array

x = Array.create('x', (n,n), 'BINARY')
x

In [ ]:
H_a1 = 0
for v in range(n):
    H_temp = 1
    for j in range(n):
        H_temp -= x[v, j]

    H_a1 += H_temp * H_temp

H_a2 = 0
for j in range(n):
    H_temp = 1
    for v in range(n):
        H_temp -= x[v, j]

    H_a2 += H_temp * H_temp

H_a3 = 0
for u in range(n):
    for v in range(n):
            k = 1
            for j in range(n):
                H_a3 +=  x[u, j] * x[v, k]

                k += 1
                if k == n: k = 0

In [ ]:
H_b = 0

for u in range(n):
    for v in range(n):
        k = 1
        for j in range(n):
            H_b +=  D[u,v] * x[u, j] * x[v, k]

            k += 1
            if k == n: k = 0

In [ ]:
H_c = 0

for v in range(n):
    if v % 2 == 0:
        for u in range(n):
            if u != v + 1 and u != v:
                H_c += x[v, 0] * x[u, 1]
    else:
        for u in range(n):
            if u != v - 1 and u != v:
                H_c += x[v, 0] * x[u, 1]

H_c

In [ ]:
H_d = 0

for u in range(n):
    for v in range(n):
        k = 1
        for j in range(n - 1):
            H_d += x[u, j] * x[v, k] * ((Q[v] - Q[u])**2)

            k += 1
            if k == n: k = 0

H_d

In [ ]:
H_A = A_NORMALIZATION * H_a1 + A_NORMALIZATION * H_a2 #+ A_NORMALIZATION * H_a3
H_B = B_NORMALIZATION * H_b
H_C = C_NORMALIZATION * H_c     # H_C is not needed anymore with H_D ranging up to j - 1
H_D = D_NORMALIZATION * H_d

H = H_A + H_B + H_D

model = H.compile()
qubo, _= model.to_qubo()

BQM = dimod.BinaryQuadraticModel.from_qubo(qubo)

In [ ]:
# allows to visualize the built matrix

Q = np.zeros((n**2,n**2))
k = np.zeros((2,2))

np.set_printoptions(linewidth=100)

for key in qubo:
    Q[int((key[0])[2]) * n + int((key[0])[5]), int((key[1])[2]) * n + int((key[0])[5])] = qubo[key]

### Util Functions

In [ ]:
def print_response_data(response):
    pos_sets = []
    response = response.lowest()
    # ------- Print results to user -------
    print('-' * 100)
    print('{:>20s}{:>42s}{:>22s}'.format('Set 1','Energy',"Count"))
    print('-' * 100)
    for sample, E, occ in response.data(fields=['sample','energy',"num_occurrences"]):
        S0 = [k for k,v in sample.items() if v == 0]
        S1 = [k for k,v in sample.items() if v == 1]
        pos_sets.append(S1)
        print('{:>30s}{:^30s}{:^15s}'.format(str(S1),str(E),str(occ)))
    
    return pos_sets

def map_variables(pos_set):
    m_set = []
    for i in range(len(pos_set)):
        x = pos_set[i].replace(']', '').split('[')
        m_set.append([int(x[1]), int(x[2])])
    
    return m_set

def return_solution(pos_solution):
    sol_num = 0
    for p_set in pos_solution:
        bool = True
        m_set = map_variables(p_set)
        s_res = sorted(m_set, key=lambda x: x[1])

        if ((s_res[0])[0] % 2) == 0:
            if ((s_res[0])[0] + 1) != (s_res[1])[0]:
                bool = False
        else:
            if ((s_res[0])[0] - 1) != (s_res[1])[0]:
                bool = False
        
        if bool:
            sol_num += 1
            for i in range(len(s_res) - 1):
                print((s_res[i])[0], end='-->')
            
            print((s_res[len(s_res) - 1])[0])

    return sol_num

def complete_return_solution(pos_solution):
    sol_num = 0
    for p_set in pos_solution:
        sol_num += 1
        m_set = map_variables(p_set)
        s_res = sorted(m_set, key=lambda x: x[1])
        for i in range(len(s_res) - 1):
            print((s_res[i])[0], end='-->')
        
        print((s_res[len(s_res) - 1])[0])
    
    return sol_num

In [ ]:
def check_solution(pos_sets):
    wrong_sol = False

    for p_set in pos_sets:
        correct = True
        m_set = map_variables(p_set)
        s_res = sorted(m_set, key=lambda x: x[1])

        for i in range(0, int(len(s_res)/2), 2):
            start_node = (s_res[i + 0])[0]
            end_node = (s_res[i + 1])[0]
            
            if(start_node % 2) == 0:
                if (start_node + 1) != end_node:
                    correct = False
            else:
                if (start_node - 1) != end_node:
                    correct = False

        if not correct:
            wrong_sol = True
            print('Wrong solution:')
            for i in range(len(s_res) - 1):
                print((s_res[i])[0], end='-->')
        
            print((s_res[len(s_res) - 1])[0])

    return wrong_sol

## SA

In [ ]:
sampler = SimulatedAnnealingSampler()

response_SA = sampler.sample(BQM, num_reads = num_reads, chain_strength = CHAIN_STRENGTH)

In [ ]:
solution_SA = print_response_data(response_SA.aggregate())

In [ ]:
if not check_solution(solution_SA):
    sol_num_SA = return_solution(solution_SA)
    print(f'There are {sol_num_SA} CORRECT solutions')

In [ ]:
all_sol_num_SA = complete_return_solution(solution_SA)
print(f'There are {all_sol_num_SA} solutions')


## QPU

In [ ]:
sampler = EmbeddingComposite(DWaveSampler(token=TOKEN))

response_QPU = sampler.sample(BQM, num_reads=num_reads, label='tsp6.txt for thesis', chain_strength=2.0)

In [ ]:
solution_QPU = print_response_data(response_QPU.aggregate())

In [ ]:
if not check_solution(solution_QPU):
    sol_num_QPU = return_solution(solution_QPU)
    print(f'There are {sol_num_QPU} solutions')

In [ ]:
all_sol_num_QPU = complete_return_solution(solution_QPU)
print(f'There are {all_sol_num_QPU} solutions')

## Plot the Energy distribution of the two solvers to compare the results

In [ ]:
import matplotlib
%matplotlib inline

from matplotlib import pyplot as plt

num_bins = 100
use_bin = 50

def histogram_energies(sampleset_SA, sampleset_QPU):
    "Plot energy histograms for both QPUs."

    fig = plt.figure(figsize=(8, 5))
    SA = sampleset_SA.record.energy
    QPU = sampleset_QPU.record.energy

    bins=np.histogram(np.hstack((SA,QPU)), bins=num_bins)[1]

    ax = fig.add_subplot(1, 1, 1)

    ax.hist(SA, bins[0:use_bin], color='g', alpha=0.4, label="SA")
    ax.hist(QPU, bins[0:use_bin], color='r', alpha=0.4, label="QPU")

    ax.set_xlabel("Energy")
    ax.set_ylabel("Samples")
    ax.legend()
    plt.show()

In [ ]:
histogram_energies(response_SA, response_QPU)

## Chain Strength Analysis - Uniform Torque Compensation

In [ ]:
import random
from dwave.embedding.chain_strength import uniform_torque_compensation

int_list = random.sample(range(500, 2000), 10)
prefactor_list = [x/1000 for x in int_list]
prefactor_list.insert(0, 1.414)

chain_strengths = []
for prefactor in prefactor_list:
    cs = uniform_torque_compensation(BQM, embedding=EmbeddingComposite(DWaveSampler(token=TOKEN)), prefactor=prefactor)
    chain_strengths.append(cs)


print(prefactor_list)
print(chain_strengths)

## Qubits

In [ ]:
# identify the embedded graph on the architecture of the solver used in the real annealing

device = DWaveSampler(token=TOKEN)
device.solver.data['id']

QUBO_graph = BQM.to_networkx_graph()
QPU_graph = device.solver.data['properties']["couplers"]

embedded_graph = minorminer.find_embedding(QUBO_graph.edges(), QPU_graph)

embedded_graph

In [ ]:
# print the number of qubit required in the embedding and the minimal and maximal length of the chains created

sublist = [values for keys, values in embedded_graph.items()]
flat_list = set(itertools.chain(*sublist))    

max_chain_length = None
min_chain_length = None

for _, chain in embedded_graph.items():
    if max_chain_length is None:
        max_chain_length = len(chain)
        min_chain_length = len(chain)

    if len(chain) > max_chain_length:
        max_chain_length = len(chain)

    if len(chain) < min_chain_length:
        min_chain_length = len(chain)

    
print("Embedding requires {} qubits and has chain lengths between {}-{}".format(len(flat_list),min_chain_length, max_chain_length))